In [29]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time
import pandas as pd
import asyncio
from abc import ABC,abstractmethod

In [26]:
class BOT(ABC):
    '''
    Abstract method for selenium bot
    '''
    @abstractmethod
    def target_html() -> str:
        pass


class RERABot(BOT):
    '''
    Clicks on ith registered Property and returns HTML
    '''
    def __init__(self):
        self.url = 'https://hprera.nic.in/PublicDashboard'
    async def target_html(self,i):
        print('Start')
        options = Options()
        options.add_argument('--headless=new')
        browser = webdriver.Chrome(options=options)
        browser.get(self.url)
        time.sleep(20)
        search = browser.find_elements(By.XPATH, "//a[@title='View Application']")

        Property = search[i]
        Property.click()
        time.sleep(6)
        html = browser.page_source
        browser.quit()

        return html

In [27]:
def parse_table(html):
    html_table = pd.read_html(html, index_col=0)[4]
    name = html_table.loc['Name'][1]
    address = html_table.loc['Permanent Address'][1].replace('Address Proof',"")
    pan = html_table.loc['PAN No.'][1].split(" ")[0]
    gstin = html_table.loc['GSTIN No.'][1].replace("GST Certificate","")

    return {
        'Name' : name,
        'Permanent Address' : address,
        'PAN No.' : pan,
        'GSTIN No.' : gstin
    }
    

In [10]:
results = await asyncio.gather(*[RERABot().target_html(i) for i in range(5)])

In [25]:
rows = [parse_table(i) for i in results ]
result = pd.DataFrame(rows)

In [30]:
result

,Name,Permanent Address,PAN No.,GSTIN No.
0,MANAVINDER SINGH,"Villette Kothi, Khalini, Shimla, Khalini, Shim...",ACLPS2284H,-NA-
1,M/S SOMA NEW TOWNS PVT.LTD.,"B-4/45, Safdarjung Enclave, New Delhi, Delhi (...",AALCS4814Q,GSTIN0123456789
2,MANAVINDER SINGH,"Villette Kothi, Khalini, Shimla, Khalini, Shim...",ACLPS2284H,-NA-
3,M/S. MAGIC RESORTS PVT. LTD.,"W-5/19, Western Avenue, Sainik Farms, New Delh...",AAHCM9235H,GSTIN0123456789
4,MS URBAN GREENS,"C/o Solutions, Near Grand Plaza Shopping Compl...",AAGFU9110M,00AAAAA0000AAAA
